# News Scraper - InfoMoney (Processamento em Lote)

Este notebook utiliza o módulo `src.scraper` para coletar notícias do InfoMoney via API.
Ele itera sobre uma lista de termos de pesquisa, salva arquivos CSV individuais e, em seguida, os consolida em um único dataset.

In [1]:
import sys
import os
import glob
from datetime import datetime
import pandas as pd

# Fix para erro de Unicode no Windows
os.environ["PYTHONUTF8"] = "1"

# Auto-reload para refletir mudanças nos scripts
%load_ext autoreload
%autoreload 2

# Add root directory to path to import src modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.scraper import get_news_from_period
from src.utils.logger import setup_logger

# Configure logger to see output in the notebook
logger = setup_logger()

## Definição de Parâmetros
Defina a lista de termos de pesquisa, a data de início da coleta e a pasta de saída.

In [2]:
SEARCH_TERMS = ["Itaú", "Dólar", "Petrobras", "Vale", "Bolsa de Valores", "Bolsa", "B3", "IBOVESPA", "Bradesco", "Economia"]
START_DATE = datetime(2025, 1, 1)
END_DATE = datetime(2025, 12, 31)
OUTPUT_DIR = os.path.join("..", "src", "dataset", "scraper", "search")

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Configurado para buscar {len(SEARCH_TERMS)} termos entre {START_DATE.date()} e {END_DATE.date()}")
print(f"Resultados serão salvos em: {os.path.abspath(OUTPUT_DIR)}")


Configurado para buscar 10 termos entre 2025-01-01 e 2025-12-31
Resultados serão salvos em: d:\dev\mba-tcc\src\dataset\scraper\search


## Execução da Coleta em Lote
Itera pelos termos, coleta dados e salva em CSV na pasta de resultados.

In [ ]:
for term in SEARCH_TERMS:
    print(f"\n{'='*50}")
    print(f"Processando termo: '{term}'")
    print(f"{'='*50}")
    
    try:
        # Execute scraper for the current term
        df = get_news_from_period(term=term, start_date=START_DATE, end_date=END_DATE)
        
        if not df.empty:
            # Generate filename
            safe_term = term.replace(" ", "_").lower()
            timestamp = datetime.now().strftime('%Y%m%d')
            filename = f"news_{safe_term}_{timestamp}.csv"
            filepath = os.path.join(OUTPUT_DIR, filename)
            
            # Save to CSV
            df.to_csv(filepath, index=False)
            print(f"\n[SUCESSO] Salvas {len(df)} notícias para '{term}' em: {filepath}")
            
            # Optional: Display first few rows
            display(df.head(3))
        else:
            print(f"\n[AVISO] Nenhuma notícia encontrada para '{term}'")
            
    except Exception as e:
        print(f"\n[ERRO] Falha ao coletar termo '{term}': {e}")


## Consolidar Resultados
Lê todos os arquivos CSV da pasta de resultados, combina-os em um único dataset e remove duplicatas (com base no link).

In [3]:
# --- 1. Execução da Coleta (Se necessário) ---
# Descomente as linhas abaixo para coletar novos dados da API
# for term in SEARCH_TERMS:
#    print(f"Processando termo: {term}")
#    get_news_from_period(term, START_DATE, END_DATE)

# --- 2. Consolidação e Filtragem ---
print("\n" + "="*50 + "\nIniciando Consolidação e Filtragem...\n" + "="*50)

# Lista de palavras-chave para filtragem
KEYWORDS = [
    "Ibovespa", "BOVA11", "Bolsa", "Ações", "Mercado", "Câmbio", "Dólar", "Juros", "Selic", "Inflação",
    "IPCA", "Banco Central", "Copom", "Fazenda", "CDI", "Petrobras", "Vale", "Itaú", "Bradesco", "Banco do Brasil",
    "B3", "Ambev", "Eletrobras", "WEG", "Suzano", "Gerdau", "Localiza", "Rumo", "Equatorial", "BTG Pactual"
]

all_files = glob.glob(os.path.join(OUTPUT_DIR, "news_*.csv"))

if all_files:
    print(f"Encontrados {len(all_files)} arquivos para consolidar.")
    
    # Read and concatenate all files
    df_list = [pd.read_csv(f) for f in all_files]
    consolidated_df = pd.concat(df_list, ignore_index=True)
    
    total_rows = len(consolidated_df)
    print(f"Total de linhas antes da desduplicação e filtragem: {total_rows}")
    
    # Drop duplicates based on 'link'
    consolidated_df = consolidated_df.drop_duplicates(subset=['link'])
    unique_rows = len(consolidated_df)
    print(f"Total únicos após desduplicação: {unique_rows}")
    
    # --- FILTRAGEM ---
    print(f"\nAplicando filtro com {len(KEYWORDS)} palavras-chave...")
    pattern = '|'.join([fr'\b{k}\b' for k in KEYWORDS]) # Adicionado \b para garantir palavra exata (evita 'privatiz-ações')
    # Converter para string e lidar com NaN antes de filtrar
    mask = consolidated_df['title'].astype(str).str.contains(pattern, case=False, na=False)
    filtered_df = consolidated_df[mask].copy()
    
    print(f"Total de notícias após filtragem: {len(filtered_df)}")
    print(f"Notícias removidas: {unique_rows - len(filtered_df)}")

    # Sort by date
    if 'date' in filtered_df.columns:
        filtered_df = filtered_df.sort_values(by='date', ascending=False)
    
    # Save consolidated file
    timestamp = datetime.now().strftime('%Y%m%d')
    # Save to parent directory of OUTPUT_DIR (dataset/infomoney)
    parent_dir = os.path.dirname(OUTPUT_DIR)
    output_file = os.path.join(parent_dir, f"consolidated_news_{timestamp}.csv")
    filtered_df.to_csv(output_file, index=False)
    
    print(f"\n[SUCESSO] Arquivo consolidado e filtrado salvo em: {output_file}")
    display(filtered_df.head())
else:
    print("Nenhum arquivo CSV encontrado na pasta de resultados para consolidar.")


Iniciando Consolidação e Filtragem...
Encontrados 10 arquivos para consolidar.
Total de linhas antes da desduplicação e filtragem: 32538
Total únicos após desduplicação: 16234

Aplicando filtro com 30 palavras-chave...
Total de notícias após filtragem: 5909
Notícias removidas: 10325

[SUCESSO] Arquivo consolidado e filtrado salvo em: ..\src\dataset\scraper\consolidated_news_20260209.csv


,date,title,link
2167,2025-12-30 19:14:13,"Ouro fecha ano com alta de 65,24%; Ibovespa é ...",https://www.infomoney.com.br/mercados/ouro-fec...
0,2025-12-30 19:03:37,O que fez o Ibovespa subir mais de 30% em reai...,https://www.infomoney.com.br/mercados/ibovespa...
1,2025-12-30 18:53:07,Ibovespa encerra ano em alta e sobe 34% em 202...,https://www.infomoney.com.br/mercados/ibovespa...
2,2025-12-30 18:39:39,4 ações saltam mais de 100% em 2025: confira o...,https://www.infomoney.com.br/mercados/4-acoes-...
2172,2025-12-30 18:39:32,5 ações caem mais de 30% em 2025; confira os d...,https://www.infomoney.com.br/mercados/5-acoes-...
